In [1]:
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
import os
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from tqdm import tqdm
sns.set_palette('Dark2')
sns.set_style({'axes.axisbelow': True, 'axes.edgecolor': '.15', 'axes.facecolor': 'white',
               'axes.grid': True, 'axes.labelcolor': '.15', 'axes.linewidth': 1.25, 
               'figure.facecolor': 'white', 'font.family': ['sans-serif'], 'grid.color': '.15',
               'grid.linestyle': ':', 'grid.alpha': .5, 'image.cmap': 'Greys', 
               'legend.frameon': False, 'legend.numpoints': 1, 'legend.scatterpoints': 1,
               'lines.solid_capstyle': 'round', 'axes.spines.right': False, 'axes.spines.top': False,  
               'text.color': '.15',  'xtick.top': False, 'ytick.right': False, 'xtick.color': '.15',
               'xtick.direction': 'out', 'xtick.major.size': 6, 'xtick.minor.size': 3,
               'ytick.color': '.15', 'ytick.direction': 'out', 'ytick.major.size': 6,'ytick.minor.size': 3})
sns.set_context('talk')

#http://phyletica.org/matplotlib-fonts/
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

In [2]:
from snapanalysis.config import OUTPUT_DIRECTORY as OUTPUT_DIRECTORY_MAIN

OUTPUT_DIRECTORY = os.path.join(OUTPUT_DIRECTORY_MAIN, 'preprocessing')
if not os.path.isdir(OUTPUT_DIRECTORY):
    os.makedirs(OUTPUT_DIRECTORY)

In [3]:
from snapanalysis.preprocessing.cleanup.main import OUTPUT_FILE as ENRICHMENT_MODEL_INPUT
from snapanalysis.models.enrichment.generate import RATIO_COLUMN
from snapanalysis.models.enrichment.generate import rotate, EnrichmentDecoposition
from snapanalysis.models.enrichment.generate import OUTPUT_FILE as ENRICHMENT_OUTPUT

In [4]:
from snapanalysis.preprocessing.pulldown_metadata import OUTPUT_FILE as PD_META_OUTPUT

In [5]:
with pd.HDFStore(PD_META_OUTPUT) as store:
    pd_names = store['/meta/names_and_types']
    pd_names = pd_names['Pull-Down name']

In [6]:
enrichment_data = pd.read_hdf(ENRICHMENT_OUTPUT, 'enrichment_data')
pulldown_data = pd.read_hdf(ENRICHMENT_MODEL_INPUT, 'pulldown_data')

protein_meta = pd.read_hdf(ENRICHMENT_MODEL_INPUT, 'protein_meta')
maxquant_meta = pd.read_hdf(ENRICHMENT_MODEL_INPUT, 'maxquant_meta')

flagged_proteins = pd.read_hdf(ENRICHMENT_MODEL_INPUT, 'flagged_proteins')

print('\n'.join(sorted(pulldown_data.columns)))

pulldown_data = pulldown_data.unstack('Direction')

pulldown_data.index = pulldown_data.index.swaplevel()
enrichment_data.index = enrichment_data.index.swaplevel()

pulldown_data = pulldown_data.sort_index()
enrichment_data = enrichment_data.sort_index()

Identification type
Intensity
Intensity H
Intensity L
Peptides
Ratio H/L
Ratio H/L (log2)
Ratio H/L count
Ratio H/L iso-count
Ratio H/L normalized
Ratio H/L normalized (log2)
Ratio H/L type
Ratio H/L variability [%]
Razor + unique peptides
Sequence coverage [%]
Unique peptides


In [7]:
print('\n'.join(sorted(enrichment_data.columns)))

Enrichment
Enrichment (imputed)
Identification type (forward)
Identification type (reverse)
Imputation type
Intensities missing
Intensity (forward)
Intensity (reverse)
Intensity H (forward)
Intensity H (reverse)
Intensity L (forward)
Intensity L (reverse)
Missing data type
Number of intensities seen
Peptides (forward)
Peptides (reverse)
Ratio H/L (forward)
Ratio H/L (log2) (forward)
Ratio H/L (log2) (reverse)
Ratio H/L (reverse)
Ratio H/L count (forward)
Ratio H/L count (reverse)
Ratio H/L iso-count (forward)
Ratio H/L iso-count (reverse)
Ratio H/L normalized (forward)
Ratio H/L normalized (log2) (adjusted, forward)
Ratio H/L normalized (log2) (adjusted, imputed, forward)
Ratio H/L normalized (log2) (adjusted, imputed, reverse)
Ratio H/L normalized (log2) (adjusted, reverse)
Ratio H/L normalized (log2) (forward)
Ratio H/L normalized (log2) (imputed, forward)
Ratio H/L normalized (log2) (imputed, reverse)
Ratio H/L normalized (log2) (reverse)
Ratio H/L normalized (reverse)
Ratio H/L typ

In [8]:
ratio_columns_processed = [
    'Ratio H/L normalized (log2) (adjusted, imputed, forward)',
    'Ratio H/L normalized (log2) (adjusted, imputed, reverse)',
    'Imputation type'
]


ratio_columns_unprocessed = [
    'Ratio H/L normalized (log2) (forward)',
    'Ratio H/L normalized (log2) (reverse)',
    'Enrichment',
    'Residual',
]

other_data_columns = [
    'Identification type (forward)',
    'Identification type (reverse)',
    'Peptides (forward)',
    'Peptides (reverse)',
    'Unique peptides (forward)',
    'Unique peptides (reverse)',
    'Razor + unique peptides (forward)',
    'Razor + unique peptides (reverse)',
    'Intensity (forward)',
    'Intensity (reverse)',
    'Intensity H (forward)',
    'Intensity H (reverse)',
    'Intensity L (forward)',
    'Intensity L (reverse)',
    'Ratio H/L type (forward)',
    'Ratio H/L type (reverse)',
    'Ratio H/L count (forward)',
    'Ratio H/L count (reverse)',
    'Ratio H/L iso-count (forward)',
    'Ratio H/L iso-count (reverse)',
    'Ratio H/L variability [%] (forward)',
    'Ratio H/L variability [%] (reverse)',
    'Sequence coverage [%] (forward)',
    'Sequence coverage [%] (reverse)'
]

protein_meta_columns = [
    'Majority protein IDs', 
    'Gene names',
    'Protein names'
]

In [9]:
pull_downs = sorted(np.unique(enrichment_data.index.get_level_values(level='Pull-Down ID')))

In [10]:
OUTPUT_FILENAME = os.path.join(OUTPUT_DIRECTORY, 'table-pulldowns.xlsx')
OUTPUT_TSV_FILENAME_TEMPLATE = os.path.join(OUTPUT_DIRECTORY, 'table-pulldowns.sheet.{}.tsv.gz')

In [12]:
color_red = '#7E3E4D'
color_blue = '#41597E'
color_white = '#F9F9F9'

def conditional_format_dict(absmax=4, 
                            reverse=False,
                            color_max=color_red,
                            color_min=color_blue,
                            color_mid=color_white): 
    
    if reverse:
        color_max, color_min = color_min, color_max
        
    
    ans = {
        'type': '3_color_scale',
        'min_type': 'num',
        'mid_type': 'num',
        'max_type': 'num',
        'min_color': color_min,
        'mid_color': color_mid,
        'max_color': color_max,
        'min_value': -absmax,
        'mid_value': 0,
        'max_value': absmax,
    }
    
    return ans

In [32]:

writer = pd.ExcelWriter(OUTPUT_FILENAME, engine='xlsxwriter')

workbook = writer.book

border_type = 2

format_header = workbook.add_format({
    'bold': True,
    'bottom': border_type,
})

format_bold = workbook.add_format({
    'bold': True,
})

format_header_border_left = workbook.add_format({
    'bold': True,
    'bottom': border_type,
    'left': border_type,
})


format_merged_header = workbook.add_format({
    'bold': True,
    'align': 'center',
    'valign': 'vcenter',
    'text_wrap': True,
})

format_merged_header_border_left = workbook.add_format({
    'bold': True,
    'align': 'center',
    'valign': 'vcenter',
    'text_wrap': True,
    'left': border_type,
})

format_merged_header_border = workbook.add_format({
    'bold': True,
    'align': 'center',
    'valign': 'vcenter',
    'text_wrap': True,
    'bottom': border_type,
})


format_merged_header_leftalign_bigger = workbook.add_format({
    'bold': True,
    'align': 'left',
    'valign': 'vcenter',
    'text_wrap': True,
    'font_size': 12,
})

format_border_left = workbook.add_format({
    'left': border_type,
})



index_sheet = workbook.add_worksheet('Index')

index_sheet.write(0, 0, 'Sheet name', format_header)
index_sheet.write(0, 1, 'Pull-Down', format_header)

for row, (pull_down, pd_name) in enumerate(pd_names.iteritems(), start=1):
    index_sheet.write(row, 0, pull_down)
    index_sheet.write(row, 1, pd_name)
    
# Write also as TSV
pd_names.to_csv(OUTPUT_TSV_FILENAME_TEMPLATE.format(
    '{:02}.{}'.format(
        1,
        'index'
    )),
    sep='\t',
)
    

index_sheet.set_column(0, 0, 10, format_bold)
index_sheet.set_column(1, 1, 60)

for i, pull_down in enumerate(pull_downs, start=2):
    
    pd_name = pd_names.loc[pull_down]
    sheet_name = pull_down
    
    
    ed = enrichment_data.loc[pull_down]
    order = ed.index

    
    output_pm = protein_meta.loc[order, protein_meta_columns].reset_index()
    output_pm.index = order
    
    output_ratios = ed.loc[order, ratio_columns_processed]
    output_ratios_unprocessed = ed.loc[order, ratio_columns_unprocessed]
    output_ratios_other_data = ed.loc[order, other_data_columns]
    
    output_ratios['Imputation type'] = output_ratios['Imputation type'].fillna('none')
    
    print(pull_down)
    
    start_row = 3
    start_col = 0
    
    last_row = start_row + len(order)

    # Write data
    
    # -- Protein metadata --------
    output_pm.to_excel(writer, 
                       startrow=start_row,
                       startcol=start_col,
                       header=False,
                       index=False,
                       sheet_name=sheet_name)
    

    # Need to do this after the first write with pandas otherise it doesn't work
    worksheet = writer.sheets[sheet_name]
    
    # Protein headers
    for col_number, label in enumerate(output_pm.columns, start=start_col):
        worksheet.merge_range(start_row-2, col_number, start_row-1, col_number, label, 
                              format_merged_header_border) 
        
    
    worksheet.merge_range(start_row-3, start_col, 
                          start_row-3, start_col + len(output_pm.columns) - 1, 
                          f'{pull_down}: {pd_name}', 
                          format_merged_header_leftalign_bigger)
    
    # Change widths a bit
    # Gene label
    worksheet.set_column(start_col, start_col, 15, format_bold)
    # Gene names
    worksheet.set_column(start_col+2, start_col+2, 15)
    # Protein names
    worksheet.set_column(start_col+3, start_col+3, 30)
    
    # -- Main ratios -------------
    first_ratio_col = start_col + len(output_pm.columns)
    worksheet.set_column(first_ratio_col, first_ratio_col, None, format_border_left)
    
    output_ratios.to_excel(writer, 
                           startrow=start_row,
                           startcol=first_ratio_col,
                           header=False,
                           index=False,
                           sheet_name=sheet_name)
    # Ratio headers
    
    # Have to hardcode headers here because the merge columns is tricky to implement
    assert np.all(output_ratios.columns == ['Ratio H/L normalized (log2) (adjusted, imputed, forward)',
                                            'Ratio H/L normalized (log2) (adjusted, imputed, reverse)',
                                            'Imputation type'])
    
    
    worksheet.merge_range(start_row-2, first_ratio_col, 
                          start_row-2, first_ratio_col + 2,
                          'Ratio H/L normalized (log2)',
                          format_merged_header_border_left)
    names = {
        'Ratio H/L normalized (log2) (adjusted, imputed, forward)': 'Forward',
        'Ratio H/L normalized (log2) (adjusted, imputed, reverse)': 'Reverse',
        'Imputation type': 'Imputation'
    }
    
    for col_number, label in enumerate(output_ratios.columns, start=first_ratio_col):
        label = names[label]
        
        if col_number == first_ratio_col:
            format_ = format_header_border_left
        else:
            format_ = format_header
            
        worksheet.write(start_row-1, col_number, label, format_)    
        
    worksheet.merge_range(start_row-3, first_ratio_col, 
                          start_row-3, first_ratio_col + len(output_ratios.columns) - 1, 
                          'Post-processed data', format_merged_header_border_left)
    
    
    
    # Ratio conditional formatting
    
    # Forward
    worksheet.conditional_format(
        start_row, first_ratio_col, 
        last_row, first_ratio_col,
        conditional_format_dict()
    )
    
    # Reverse
    worksheet.conditional_format(
        start_row, first_ratio_col + 1, 
        last_row, first_ratio_col + 1,
        conditional_format_dict(reverse=True)
    )
    
    # -- Raw ratios -------------------
    
    # Have to hardcode headers here because the merge columns is tricky to implement
    assert np.all(output_ratios_unprocessed.columns == ['Ratio H/L normalized (log2) (forward)',
                                                        'Ratio H/L normalized (log2) (reverse)',
                                                        'Enrichment',
                                                        'Residual'])
    
    first_ratio_unprocessed_col = first_ratio_col + len(output_ratios.columns)
    worksheet.set_column(first_ratio_unprocessed_col, first_ratio_unprocessed_col, None, format_border_left)
    
    output_ratios_unprocessed.to_excel(writer, 
                                       startrow=start_row,
                                       startcol=first_ratio_unprocessed_col,
                                       header=False,
                                       index=False,
                                       sheet_name=sheet_name)
    
    # Raw ratio columns
    
    worksheet.merge_range(start_row-3, first_ratio_unprocessed_col, 
                          start_row-3, first_ratio_unprocessed_col + len(output_ratios_unprocessed.columns) - 1, 
                          'Raw data', format_merged_header_border_left)
    
    worksheet.merge_range(start_row-2, first_ratio_unprocessed_col, 
                         start_row-2, first_ratio_unprocessed_col + 1,
                        'Ratio H/L normalized (log2)',
                         format_merged_header_border_left)
    
    worksheet.merge_range(start_row-2, first_ratio_unprocessed_col + 2, 
                         start_row-2, first_ratio_unprocessed_col + 3,
                        'Enrichment decomposition',
                         format_merged_header)
    
    names = {
        'Ratio H/L normalized (log2) (forward)': 'Forward',
        'Ratio H/L normalized (log2) (reverse)': 'Reverse'
    }
    
    for col_number, label in enumerate(output_ratios_unprocessed.columns, 
                                       start=first_ratio_unprocessed_col):
        label = names.get(label, label)
        
        if col_number == first_ratio_unprocessed_col:
            format_ = format_header_border_left
        else:
            format_ = format_header
            
        worksheet.write(start_row-1, col_number, label, format_)    
        
    
    # -- Other useful data ---
    first_other_data_col = first_ratio_unprocessed_col + len(output_ratios_unprocessed.columns)
    worksheet.set_column(first_other_data_col, first_other_data_col, None, format_border_left)
    
    output_ratios_other_data.to_excel(writer, 
                                      startrow=start_row,
                                      startcol=first_other_data_col,
                                      header=False,
                                      index=False,
                                      sheet_name=sheet_name)
    
    ## Also save as TSV
    output_ratios_processed_for_tsv = output_ratios.copy()
    output_ratios_processed_for_tsv.columns = [f'Processed {c}' if c.startswith('Ratio') else c for c in output_ratios_processed_for_tsv.columns]
    
    output_ratios_unprocessed_for_tsv = output_ratios_unprocessed.copy()
    output_ratios_unprocessed_for_tsv.columns = [f'Raw {c}' if c.startswith('Ratio') else c for c in output_ratios_unprocessed_for_tsv.columns]
    
    df_all_together_for_tsv = output_pm.join(output_ratios_processed_for_tsv).join(output_ratios_unprocessed_for_tsv).join(output_ratios_other_data)
    
    # Write also as TSV
    df_all_together_for_tsv.to_csv(OUTPUT_TSV_FILENAME_TEMPLATE.format(
        '{:02}.{}'.format(
            i,
            pull_down.lower()
        )),
        sep='\t', index=False,
    )

    
    # headers
    worksheet.merge_range(start_row-3, first_other_data_col, 
                          start_row-3, first_other_data_col + len(output_ratios_other_data.columns) - 1, 
                          'Metadata', format_merged_header_border_left)
    
    cs = output_ratios_other_data.columns
    
    col_i = first_other_data_col
    for forward_label, reverse_label in zip(cs[::2], cs[1::2]):
        
        assert forward_label.endswith('(forward)')
        assert reverse_label.endswith('(reverse)')
        
        assert forward_label[:-len(' (forward)')] == reverse_label[:-len(' (reverse)')]
        
        common_label = forward_label[:-len(' (forward)')]
        
        
        if col_i == first_other_data_col:
            format_ = format_header_border_left
            
            fm_merged = format_merged_header_border_left
        else:
            format_ = format_header
            fm_merged = format_merged_header
        
        worksheet.merge_range(start_row-2, col_i, 
                              start_row-2, col_i + 1,
                              common_label,
                              fm_merged)
        
        worksheet.write(start_row-1, col_i, 'Forward', format_) 
        worksheet.write(start_row-1, col_i+1, 'Reverse', format_header) 
        
        col_i += 2
        
    worksheet.set_column(col_i, col_i, None, format_border_left)
    
    worksheet.autofilter(start_row-1, start_col, last_row, col_i-1)
    worksheet.freeze_panes(start_row, first_ratio_unprocessed_col)
#     break
    
    
writer.save()


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:76: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.


H01
H01M
H02
H03
H03M
H04
H04M
H05
H06
H07
H07M
H08
H08M
H09
H10
H11
H12
H13
H14
H15
H16
H17
H18
H19
H20
H21
H22
H23
H24
H25
H26
H27M
H28
H29
H30
H31
H32
H33
H34
H35
H36
H37
H38
H39
H39M
H40
H41
H42
H43
H44
H45
H46
H46M
H47
H47M


In [23]:
output_ratios.columns

Index(['Ratio H/L normalized (log2) (adjusted, imputed, forward)',
       'Ratio H/L normalized (log2) (adjusted, imputed, reverse)',
       'Imputation type'],
      dtype='object')

In [24]:
output_ratios_unprocessed.columns

Index(['Ratio H/L normalized (log2) (forward)',
       'Ratio H/L normalized (log2) (reverse)', 'Enrichment', 'Residual'],
      dtype='object')

In [26]:
output_pm

,Gene label,Majority protein IDs,Gene names,Protein names
Gene label,,,,
A0A087X222,A0A087X222,A0A087X222,,NaN
A0A0C4DFX4,A0A0C4DFX4,A0A0C4DFX4,,NaN
A0A0C4DGP2,A0A0C4DGP2,A0A0C4DGP2,,NaN
A0A0C4DGP5,A0A0C4DGP5,A0A0C4DGP5,,NaN
AAAS,AAAS,F8VZ44;Q9NRG9;H3BU82,AAAS,Aladin
AATF,AATF,Q9NY61,AATF,Protein AATF
ABCF2,ABCF2,Q9UG63;C9JHK9;C9JZV3,ABCF2,ATP-binding cassette sub-family F member 2
ABHD12,ABHD12,Q8N2K0,ABHD12,Monoacylglycerol lipase ABHD12
ABRAXAS1,ABRAXAS1,D6REL5;Q6UWZ7,ABRAXAS1,BRCA1-A complex subunit Abraxas
